출처, 참고 : https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/how_tos/using_gpu/

텐서플로우에서는 cpu뿐만 아니라 gpu의 지원도 됩니다. 그래서 계산 디바이스를 구분하여 사용하기 위해 tf.device를 제공합니다.<br />
이를 사용하여 연산이나 텐서를 cpu나 gpu에 각각 할당할 수 있습니다. '/cpu:0','/gpu:0','/gpu:1'과 같이 배정이 된다.<br />
여기서 숫자는 디바이스 갯수를 의미한다. 만약 컴퓨터에 gpu가 여러 개 설치되어 있으면 '/gpu:0', '/gpu:1' , ... '/gpu:n'과 같이 지정

In [1]:
import tensorflow as tf

print(tf.__version__)

1.2.1


수동으로 device(cpu)에 배치하기

In [2]:
with tf.device('/cpu:0'):
    a = tf.constant([1],name = 'a')
    b = tf.constant([2],name = 'b')
    c = tf.add(a,b,name = 'c')   

어디에 배치되어 실행 되었는지 tf.Session에서 log_device_placement라는 옵션을 통해 알 수 있다.

In [3]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.run(c))

[3]


근데 출력이 jupyter notebook 안되네요... <br />
그래서 다른 방법을 찾아보았습니다. <br />
출처 : https://stackoverflow.com/questions/39677168/tensorflow-documentations-example-code-on-logging-device-placement-doesnt-pr 
jupyter notebook으로는 출력이 되지 않아 다른 옵션들을 사용해야합니다.<br />
다음과 같이 설정하고 출력을 metadata.partition_graphs와 같이 합니다. 그렇다면 정해준 tensor의 name과 함께<br />
device: "/job:localhost/replica:0/task:0/cpu:0"와 같이 cpu에 할당된 것을 볼 수 있습니다.

In [4]:
options = tf.RunOptions(output_partition_graphs=True)
metadata = tf.RunMetadata()
print(sess.run([a,b,c],options = options,  run_metadata=metadata))
print(metadata.partition_graphs)

[array([1]), array([2]), array([3])]
[node {
  name: "a"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/cpu:0"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 1
          }
        }
        int_val: 1
      }
    }
  }
}
node {
  name: "_retval_a_0_0"
  op: "_Retval"
  input: "a"
  device: "/job:localhost/replica:0/task:0/cpu:0"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "index"
    value {
      i: 0
    }
  }
}
node {
  name: "b"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/cpu:0"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 1
          }
        }
        int_val: 2
      }
    }
  }
}
node {
  name: "_retval_b_0_

In [5]:
with tf.device('/gpu:0'):
    a_gpu = tf.constant([1],name = 'a_gpu')
    b_gpu = tf.constant([2],name = 'b_gpu')
    c_gpu = tf.add(a,b,name = 'c_gpu')   

이번에는 gpu에 할당해보겠습니다.<br />
device: "/job:localhost/replica:0/task:0/gpu:0"라고 나오는 군요...참고로 '/gpu:1'이라고 하면 저는 안 됩니다. <br />
왜냐하면 gpu가 하나뿐이라 '/gpu:1'라고 입력을 하면 오류가 나옵니다. 

In [6]:
options = tf.RunOptions(output_partition_graphs=True)
metadata = tf.RunMetadata()
print(sess.run([a_gpu,b_gpu,c_gpu],options = options,  run_metadata=metadata))
print(metadata.partition_graphs)

[array([1]), array([2]), array([3])]
[node {
  name: "b/_3"
  op: "_HostRecv"
  device: "/job:localhost/replica:0/task:0/gpu:0"
  attr {
    key: "client_terminated"
    value {
      b: false
    }
  }
  attr {
    key: "recv_device"
    value {
      s: "/job:localhost/replica:0/task:0/gpu:0"
    }
  }
  attr {
    key: "send_device"
    value {
      s: "/job:localhost/replica:0/task:0/cpu:0"
    }
  }
  attr {
    key: "send_device_incarnation"
    value {
      i: 1
    }
  }
  attr {
    key: "tensor_name"
    value {
      s: "edge_4_b"
    }
  }
  attr {
    key: "tensor_type"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "a/_1"
  op: "_HostRecv"
  device: "/job:localhost/replica:0/task:0/gpu:0"
  attr {
    key: "client_terminated"
    value {
      b: false
    }
  }
  attr {
    key: "recv_device"
    value {
      s: "/job:localhost/replica:0/task:0/gpu:0"
    }
  }
  attr {
    key: "send_device"
    value {
      s: "/job:localhost/replica:0/task:0/cpu:0"
  